In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from keras.datasets import mnist
# Used for Confusion Matrix
from sklearn import metrics
# Used for calculating result and accuracy
from sklearn.metrics import f1_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
# Used for scaling image data
from sklearn import preprocessing
 
from sklearn.model_selection import train_test_split
 
import seaborn as sns
from sklearn.metrics import confusion_matrix
 
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
 
%matplotlib inline

ok


In [2]:
def get_data(input_shape):
    num_classes = 10
    (train_img, train_lbl), (test_img, test_lbl) = mnist.load_data()
    input_dim = 28*28 
    train_img = train_img.reshape(60000, input_dim) 
    test_img = test_img.reshape(10000, input_dim) 
    train_img = train_img.astype("float32") / 255
    test_img = test_img.astype("float32") / 255
    return train_img, train_lbl, test_img, test_lbl

In [3]:
def plot_result(y_test, y_test_pred, test_f1s, test_f1s_avg, pseudo_labels, epoch):
    fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize=(25,25))
    ax1.plot(range(epoch), test_f1s)
    ax1.set_ylabel('f1 Score')
    ax1.legend(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], loc="lower right")
    ax2.plot(range(epoch), test_f1s_avg, )
    ax2.set_ylabel('Average f1 Score')
    ax2.legend("Average", loc="lower right")
    ax3.bar(x=range(epoch), height=pseudo_labels)
    ax3.set_ylabel('Pseudo-Labels Created')
    ax3.set_xlabel('# Iterations')
 
    # View confusion matrix after self-training
    cf_matrix = confusion_matrix(y_test, y_test_pred, normalize='true')
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(cf_matrix, annot=True, 
                fmt='.2f', cmap='Blues')

In [4]:
def define_dbn_model(input_shape, W, Bh):
    dense_l0 = {}

    if W is not None and Bh is not None:
        dense_l0['weights'] = (W, Bh)

    model = keras.models.Sequential(
        [
            keras.layers.Dense(500,input_shape=(784,), activation='relu', kernel_regularizer=keras.regularizers.L2(1e-5), kernel_initializer=keras.initializers.GlorotUniform(seed=1111), **dense_l0),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(500, activation='relu', kernel_regularizer=keras.regularizers.L2(1e-5), kernel_initializer=keras.initializers.GlorotUniform(seed=1111)),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(500, activation='relu', kernel_regularizer=keras.regularizers.L2(1e-5), kernel_initializer=keras.initializers.GlorotUniform(seed=1111)),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(10, activation='softmax', kernel_initializer=keras.initializers.GlorotUniform(seed=2222))
        ]
    )
 
    model.compile(
        optimizer=keras.optimizers.Adam(0.001),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

In [ ]:
import sys
import time


class Stopwatch(object):
    def __init__(self, verbose=False):
        self.verbose = verbose
        if sys.platform == 'win32':
            # on Windows, the best timer is time.clock()
            self._timer_func = time.clock
        else:
            # on most other platforms, the best timer is time.time()
            self._timer_func = time.time
        self.reset()

    def __enter__(self, verbose=False):
        return self.start()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.stop()
        return self.elapsed()

    def start(self):
        if not self._is_running:
            self._start = self._timer_func()
            self._is_running = True
        return self

    def stop(self):
        if self._is_running:
            self._total += (self._timer_func() - self._start)
            self._is_running = False
        return self

    def elapsed(self):
        if self._is_running:
            now = self._timer_func()
            self._total += (now - self._start)
            self._start = now
        if self.verbose:
            print("Elapsed time: {0:.3f} sec".format(self._total))
        return self._total

    def reset(self):
        self._start = 0.
        self._total = 0.
        self._is_running = False
        return self

In [ ]:
def one_hot(y, n_classes=None):
    n_classes = n_classes or np.max(y) + 1
    return np.eye(n_classes)[y]

In [ ]:
def self_train_with_pretrain(num_pseudo: int, batchsize: int, epochs: int):
    input_shape=(28, 28, 1)
    X_train, y_train, X_test, y_test = get_data(input_shape=input_shape)
    train_f1s = []
    test_f1s = []
    train_f1s_avg = []
    test_f1s_avg = []
    pseudo_labels = []
    high_prob_counter = 1
    total_pseudo_labelled = 0
    total_prelabelled_added = 0
    X_train_s = []
    y_train_s = []
    count = 0

    (W, _, Bh) = rbm.get_weights()
        with open('../RBM_weights/W.npy', 'wb') as f:
            np.save(f,W)
        with open('../RBM_weights/Bh.npy', 'wb') as f:
            np.save(f,Bh)

    for i in range(10):
        for j in range(60000):
            if y_train[j]==i:
                count+=1
                y_train_s.append(y_train[j])
                X_train_s.append(X_train[j])
                if count > ((num_pseudo/10)-1):
                    count = 0
                    break
    X_train_s = np.array(X_train_s)
    y_train_s = np.array(y_train_s)
 
    X_plabelled = np.empty((0,784))
    y_plabelled = np.empty(0)
    unlabelled_missed = np.empty((0,784))
 
    i = 0
    #pretrain 600 labelled
    model_pretrained = define_dbn_model(input_shape, W, Bh)
    with Stopwatch(verbose=True) as s:
            early_stopping = keras.callbacks.EarlyStopping(monitor='loss', patience=12, verbose=2)
            reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, verbose=2, patience=6, min_lr=1e-5)
            callbacks = [early_stopping, reduce_lr]
            try:
                model_pretrained.fit(X_train_s, y_train_s,
                        epochs=epochs,
                        batch_size=batch_size,
                        shuffle=False,
                        callbacks=callbacks)
            except KeyboardInterrupt:
                pass

    print(f"{num_pseudo} pre-labelled data is training (pre-trained model is ready)...")
 
    X_train_s = np.empty((0,784))
    y_train_s = np.empty(0)
 
    total_prelabelled_added += num_pseudo
    i += 1
    epoch = 0
    num_new_labeled = 0
 
    while high_prob_counter > 0:
        high_prob_counter = 0
        epoch += 1
 
        print(f"Iteration: {epoch}")
 
        model = model_pretrained
 
        if X_plabelled.shape[0] > num_pseudo:
 
              X_train_s = np.append(X_train_s, X_plabelled[:num_pseudo], axis=0)
              y_train_s = np.append(y_train_s, y_plabelled[:num_pseudo])
 
              X_plabelled = X_plabelled[num_pseudo:]
              y_plabelled = y_plabelled[num_pseudo:]
 
              num_new_labeled += 1
              print(f"{num_pseudo*num_new_labeled} new labelled data is training...")

              with Stopwatch(verbose=True) as s:
                  early_stopping = keras.callbacks.EarlyStopping(monitor='loss', patience=12, verbose=2)
                  reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, verbose=2, patience=6, min_lr=1e-5)
                  callbacks = [early_stopping, reduce_lr]
                  try:
                      model.fit(X_train_s, y_train_s,
                              epochs=epochs,
                              batch_size=batch_size,
                              shuffle=False,
                              callbacks=callbacks)
                  except KeyboardInterrupt:
                      pass
              #model.fit(X_train_s, y_train_s, batch_size=batch_size, epochs=epochs, verbose=0)
 
        y_test_pred = np.argmax(model.predict(X_test),axis=1)
        y_train_pred = np.argmax(model.predict(X_train),axis=1)
 
        train_f1 = f1_score(y_train, y_train_pred, average=None)
        test_f1 = f1_score(y_test, y_test_pred, average=None)
 
        print(f"Train f1 Score: {train_f1}")
        print(f"Test f1 Score: {test_f1}")
 
        train_f1s.append(train_f1)
        test_f1s.append(test_f1)
 
        
        train_f1_avg = f1_score(y_train, y_train_pred, average='macro')
        test_f1_avg = f1_score(y_test, y_test_pred, average='macro')
 
        print(f"Train f1 Score: {train_f1_avg}")
        print(f"Test f1 Score: {test_f1_avg}")
 
        train_f1s_avg.append(train_f1_avg)
        test_f1s_avg.append(test_f1_avg)
 
 
        
        num_unlabaled = num_pseudo * 1
        if num_unlabaled*(i+1) < 60000:
            unlabelled_splitted = X_train[(num_unlabaled*(i)):(num_unlabaled*(i+1))]
            i += 1
        elif unlabelled_missed.shape[0] > num_unlabaled:
            unlabelled_splitted = unlabelled_missed[:num_unlabaled]
            unlabelled_missed = unlabelled_missed[num_unlabaled:]
        else:
            unlabelled_splitted = unlabelled_missed
            unlabelled_missed = np.empty((0,784))
 
        if unlabelled_splitted.shape[0] == 0:
            print("Whole dataset is observed. Nothing remained.")
            break
        unlabeled_pred_prob = model.predict_proba(unlabelled_splitted)
 
        print(f"Now predicting labels for unlabeled data...")
 
        high_prob_idx = []
 
        for j in range(unlabeled_pred_prob.shape[0]):
            max = 0
            pseudo_label = 0
            pred_prob = unlabeled_pred_prob[j]
            for k in range(pred_prob.shape[0]):
                if pred_prob[k] > max:
                    max = pred_prob[k]
                    pseudo_label = k
            if max > 0.95:
                #print(j, max)
                high_prob_counter += 1
                high_prob_idx.append(j)
                X_plabelled = np.append(X_plabelled, [unlabelled_splitted[j]], axis=0)
                y_plabelled = np.append(y_plabelled, pseudo_label)
            else:
                unlabelled_missed = np.append(unlabelled_missed, [unlabelled_splitted[j]], axis=0)
 
        print(f"{high_prob_counter} high-probability unlabelled predictions is labelled and added to next train dataset.")
 
        print(f"{unlabelled_missed.shape[0]} unlabelled instances remained to be predict in next iteration")
 
        print(f"{X_plabelled.shape[0]} labelled instances are going to be added to train dataset in next iteration.")
 
        
        total_pseudo_labelled += high_prob_counter
        pseudo_labels.append(high_prob_counter)
   
    
    print("*** Model's Learning is finished. ***")
    print(f"In the end, {total_prelabelled_added} pre-labelled images and {total_pseudo_labelled} pseudo-labelled images added to train dataset")
    return y_test, y_test_pred, test_f1s, test_f1s_avg, pseudo_labels, epoch

In [ ]:
#HyperParams
batch_size = 128
epochs = 100 
num_pseudo = 600
y_test, y_test_pred, test_f1s, test_f1s_avg, pseudo_labels, epoch = self_train_with_pretrain(num_pseudo, batch_size, epochs)

In [ ]:
plot_result(y_test, y_test_pred, test_f1s, test_f1s_avg, pseudo_labels, epoch)